### Setup

In [1]:
# import dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# load in csv files
world_cups_csv = "Data/WorldCups.csv"
wc_matches_csv = "Data/WorldCupMatches.csv"
wc_players_csv = "Data/WorldCupPlayers.csv"

In [3]:
# read the csv files to data frames
world_cups_df = pd.read_csv(world_cups_csv)
wc_matches_df = pd.read_csv(wc_matches_csv)
wc_players_df = pd.read_csv(wc_players_csv)

### World Cups Data

In [4]:
world_cups_df['Attendance'] = world_cups_df['Attendance'].str.replace('.','').astype(int)

In [5]:
world_cups_df.rename(columns={'Runners-Up':'Second'}, inplace=True)

In [6]:
world_cups_df.head()

,Year,Country,Winner,Second,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1045246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768607


### World Cup Matches

In [7]:
wc_matches_df = wc_matches_df.dropna().reset_index(drop=True)

In [8]:
wc_matches_df = wc_matches_df.drop_duplicates('MatchID').reset_index(drop=True)

In [9]:
wc_matches_df = wc_matches_df.apply(pd.to_numeric, errors='ignore', downcast='integer')

In [10]:
wc_matches_df['Datetime'] = pd.to_datetime(wc_matches_df['Datetime'])

In [11]:
wc_matches_df.insert(2,'Date', '')
wc_matches_df.insert(3,'Time', '')
wc_matches_df['Date'] = [d.date() for d in wc_matches_df['Datetime']]
wc_matches_df['Time'] = [d.time() for d in wc_matches_df['Datetime']]
del wc_matches_df['Datetime']

In [12]:
wc_matches_df.insert(6,'Home Team',wc_matches_df['Home Team Initials'])
wc_matches_df.insert(10,'Away Team',wc_matches_df['Away Team Initials'])
del wc_matches_df['Home Team Name']
del wc_matches_df['Away Team Name']
del wc_matches_df['Away Team Initials']
del wc_matches_df['Home Team Initials']

In [13]:
del wc_matches_df['Half-time Home Goals']
del wc_matches_df['Half-time Away Goals']
del wc_matches_df['RoundID']

In [14]:
wc_matches_df.rename(columns={'MatchID':'MatchID_old'}, inplace=True)
wc_matches_df.insert(0, 'MatchID', wc_matches_df['MatchID_old'])
del wc_matches_df['MatchID_old']

In [15]:
wc_matches_df.rename(columns={'Home Team':'Home_Team',
                             'Home Team Goals':'Home_Team_Goals',
                             'Away Team Goals':'Away_Team_Goals',
                             'Away Team':'Away_Team',
                             'Win conditions':'Win_Conditions',
                             'Assistant 1':'Assistant_1',
                             'Assistant 2':'Assistant_2',}, inplace=True)

In [16]:
wc_matches_df.head()

,MatchID,Year,Date,Time,Stage,Stadium,City,Home_Team,Home_Team_Goals,Away_Team_Goals,Away_Team,Win_Conditions,Attendance,Referee,Assistant_1,Assistant_2
0,1096,1930,1930-07-13,15:00:00,Group 1,Pocitos,Montevideo,FRA,4,1,MEX,,4444,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA)
1,1090,1930,1930-07-13,15:00:00,Group 4,Parque Central,Montevideo,USA,3,0,BEL,,18346,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI)
2,1093,1930,1930-07-14,12:45:00,Group 2,Parque Central,Montevideo,YUG,2,1,BRA,,24059,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA)
3,1098,1930,1930-07-14,14:50:00,Group 3,Pocitos,Montevideo,ROU,3,1,PER,,2549,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU)
4,1085,1930,1930-07-15,16:00:00,Group 1,Parque Central,Montevideo,ARG,1,0,FRA,,23409,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU)


### World Cup Players

In [17]:
del wc_players_df['Coach Name']
del wc_players_df['Line-up']
del wc_players_df['Shirt Number']
del wc_players_df['Position']

In [18]:
wc_players_df = wc_players_df[wc_players_df['MatchID'] != 300186460].reset_index(drop=True)

In [19]:
wc_players_df['Event'] = wc_players_df['Event'].fillna('')

In [20]:
wc_players_df['Goals'] = 0
wc_players_df['Own_Goals'] = 0
wc_players_df['Yellow_Card'] = 0
wc_players_df['Red_Card'] = 0
wc_players_df['Second_Yellow_Card'] = 0
wc_players_df['Penalty'] = 0
wc_players_df['Missed_Penalty'] = 0

In [21]:
for index, row in wc_players_df.iterrows():
    events = row['Event'].split(' ')
    for event in events:
        if len(event) != 0:
            if event[0] == 'G':
                wc_players_df.loc[index,'Goals'] += 1
            elif event[0:2] == 'OG':
                wc_players_df.loc[index,'Own_Goals'] += 1
            elif event[0] == 'Y':
                wc_players_df.loc[index,'Yellow_Card'] += 1
            elif event[0] == 'R':
                wc_players_df.loc[index,'Red_Card'] += 1
            elif event[0:2] == 'SY':
                wc_players_df.loc[index,'Second_Yellow_Card'] += 1
            elif event[0] == 'P':
                wc_players_df.loc[index,'Penalty'] += 1
            elif event[0:2] == 'MP':
                wc_players_df.loc[index,'Missed_Penalty'] += 1

In [22]:
del wc_players_df['Event']
del wc_players_df['RoundID']

In [23]:
wc_players_df.insert(0, 'Player_MatchID', wc_players_df['Player Name']+'_'+wc_players_df['MatchID'].astype(str))

In [24]:
wc_players_df = wc_players_df.drop_duplicates('Player_MatchID').reset_index(drop=True)

In [25]:
wc_players_df.rename(columns={'Team Initials':'Team_Initials','Player Name':'Player_Name'},inplace=True)

In [26]:
wc_players_df.head(7)

,Player_MatchID,MatchID,Team_Initials,Player_Name,Goals,Own_Goals,Yellow_Card,Red_Card,Second_Yellow_Card,Penalty,Missed_Penalty
0,Alex THEPOT_1096,1096,FRA,Alex THEPOT,0,0,0,0,0,0,0
1,Oscar BONFIGLIO_1096,1096,MEX,Oscar BONFIGLIO,0,0,0,0,0,0,0
2,Marcel LANGILLER_1096,1096,FRA,Marcel LANGILLER,1,0,0,0,0,0,0
3,Juan CARRENO_1096,1096,MEX,Juan CARRENO,1,0,0,0,0,0,0
4,Ernest LIBERATI_1096,1096,FRA,Ernest LIBERATI,0,0,0,0,0,0,0
5,Rafael GARZA_1096,1096,MEX,Rafael GARZA,0,0,0,0,0,0,0
6,Andre MASCHINOT_1096,1096,FRA,Andre MASCHINOT,2,0,0,0,0,0,0


### Unique Players

In [27]:
# when grouping with team initials and player name together to keep the country in the table,
    # duplicates arise due to different teams like Germany vs French Germany
# unique_players_df = wc_players_df.groupby(['Player_Name','Team_Initials'], as_index=False).sum()
# del unique_players_df['MatchID']

In [28]:
unique_players_df = wc_players_df.groupby(['Player_Name'], as_index=False).sum()
del unique_players_df['MatchID']

In [29]:
unique_players_df = unique_players_df.sort_values('Goals', ascending=False).reset_index(drop=True)

In [30]:
unique_players_df.head()

,Player_Name,Goals,Own_Goals,Yellow_Card,Red_Card,Second_Yellow_Card,Penalty,Missed_Penalty
0,KLOSE,16,0,3,1,0,0,0
1,RONALDO,16,0,3,0,0,1,0
2,Just FONTAINE,13,0,0,0,0,0,0
3,Gerd MUELLER,13,0,2,0,0,1,0
4,PEL� (Edson Arantes do Nascimento),12,0,0,0,0,0,0


### Push DataFrames to SQL Database and Display Them

In [31]:
rds_connection_string = "postgres:postgres@localhost:5432/FIFA_World_Cup"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [32]:
# Confirm tables
engine.table_names()

['world_cups', 'wc_matches', 'wc_players', 'unique_players']

In [33]:
world_cups_df.to_sql(name="world_cups", con=engine, if_exists='append', index=True)

In [34]:
wc_matches_df.to_sql(name="wc_matches", con=engine, if_exists='append', index=True)

In [35]:
unique_players_df.to_sql(name="unique_players", con=engine, if_exists='append', index=True)

In [36]:
wc_players_df.to_sql(name="wc_players", con=engine, if_exists='append', index=True)

In [37]:
pd.read_sql_query('select * from world_cups', con=engine).head()

,index,Year,Country,Winner,Second,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance,data_date
0,0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590549,2019-08-21 20:17:09.205611
1,1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363000,2019-08-21 20:17:09.205611
2,2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375700,2019-08-21 20:17:09.205611
3,3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1045246,2019-08-21 20:17:09.205611
4,4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768607,2019-08-21 20:17:09.205611


In [38]:
pd.read_sql_query('select * from wc_matches', con=engine).head()

,index,MatchID,Year,Date,Time,Stage,Stadium,City,Home_Team,Home_Team_Goals,Away_Team_Goals,Away_Team,Win_Conditions,Attendance,Referee,Assistant_1,Assistant_2,data_date
0,0,1096,1930,1930-07-13,15:00:00,Group 1,Pocitos,Montevideo,FRA,4,1,MEX,,4444,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),2019-08-21 20:17:09.310181
1,1,1090,1930,1930-07-13,15:00:00,Group 4,Parque Central,Montevideo,USA,3,0,BEL,,18346,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),2019-08-21 20:17:09.310181
2,2,1093,1930,1930-07-14,12:45:00,Group 2,Parque Central,Montevideo,YUG,2,1,BRA,,24059,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),2019-08-21 20:17:09.310181
3,3,1098,1930,1930-07-14,14:50:00,Group 3,Pocitos,Montevideo,ROU,3,1,PER,,2549,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),2019-08-21 20:17:09.310181
4,4,1085,1930,1930-07-15,16:00:00,Group 1,Parque Central,Montevideo,ARG,1,0,FRA,,23409,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),2019-08-21 20:17:09.310181


In [39]:
pd.read_sql_query('select * from unique_players', con=engine).head()

,index,Player_Name,Goals,Own_Goals,Yellow_Card,Red_Card,Second_Yellow_Card,Penalty,Missed_Penalty,data_date
0,0,KLOSE,16,0,3,1,0,0,0,2019-08-21 20:17:10.057287
1,1,RONALDO,16,0,3,0,0,1,0,2019-08-21 20:17:10.057287
2,2,Just FONTAINE,13,0,0,0,0,0,0,2019-08-21 20:17:10.057287
3,3,Gerd MUELLER,13,0,2,0,0,1,0,2019-08-21 20:17:10.057287
4,4,PEL� (Edson Arantes do Nascimento),12,0,0,0,0,0,0,2019-08-21 20:17:10.057287


In [40]:
pd.read_sql_query('select * from wc_players', con=engine).head()

,index,Player_MatchID,MatchID,Team_Initials,Player_Name,Goals,Own_Goals,Yellow_Card,Red_Card,Second_Yellow_Card,Penalty,Missed_Penalty,data_date
0,0,Alex THEPOT_1096,1096,FRA,Alex THEPOT,0,0,0,0,0,0,0,2019-08-21 20:17:14.663174
1,1,Oscar BONFIGLIO_1096,1096,MEX,Oscar BONFIGLIO,0,0,0,0,0,0,0,2019-08-21 20:17:14.663174
2,2,Marcel LANGILLER_1096,1096,FRA,Marcel LANGILLER,1,0,0,0,0,0,0,2019-08-21 20:17:14.663174
3,3,Juan CARRENO_1096,1096,MEX,Juan CARRENO,1,0,0,0,0,0,0,2019-08-21 20:17:14.663174
4,4,Ernest LIBERATI_1096,1096,FRA,Ernest LIBERATI,0,0,0,0,0,0,0,2019-08-21 20:17:14.663174


### SQL Queries

In [41]:
players_matches_df = pd.read_sql_query('SELECT p."MatchID", m."Year", m."Date", m."Stage", m."Attendance", p."Player_MatchID", p."Player_Name", p."Goals", p."Own_Goals", m."Referee", p."Yellow_Card", p."Red_Card", p."Second_Yellow_Card", p."Penalty", p."Missed_Penalty", m."Win_Conditions" FROM wc_players AS p INNER JOIN wc_matches AS m ON p."MatchID"=m."MatchID"', con=engine)
players_matches_df.head(10)

,MatchID,Year,Date,Stage,Attendance,Player_MatchID,Player_Name,Goals,Own_Goals,Referee,Yellow_Card,Red_Card,Second_Yellow_Card,Penalty,Missed_Penalty,Win_Conditions
0,1096,1930,1930-07-13,Group 1,4444,Alex THEPOT_1096,Alex THEPOT,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,
1,1096,1930,1930-07-13,Group 1,4444,Oscar BONFIGLIO_1096,Oscar BONFIGLIO,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,
2,1096,1930,1930-07-13,Group 1,4444,Marcel LANGILLER_1096,Marcel LANGILLER,1,0,LOMBARDI Domingo (URU),0,0,0,0,0,
3,1096,1930,1930-07-13,Group 1,4444,Juan CARRENO_1096,Juan CARRENO,1,0,LOMBARDI Domingo (URU),0,0,0,0,0,
4,1096,1930,1930-07-13,Group 1,4444,Ernest LIBERATI_1096,Ernest LIBERATI,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,
5,1096,1930,1930-07-13,Group 1,4444,Rafael GARZA_1096,Rafael GARZA,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,
6,1096,1930,1930-07-13,Group 1,4444,Andre MASCHINOT_1096,Andre MASCHINOT,2,0,LOMBARDI Domingo (URU),0,0,0,0,0,
7,1096,1930,1930-07-13,Group 1,4444,Hilario LOPEZ_1096,Hilario LOPEZ,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,
8,1096,1930,1930-07-13,Group 1,4444,Etienne MATTLER_1096,Etienne MATTLER,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,
9,1096,1930,1930-07-13,Group 1,4444,Dionisio MEJIA_1096,Dionisio MEJIA,0,0,LOMBARDI Domingo (URU),0,0,0,0,0,


In [42]:
team_attend_matches_df = pd.read_sql_query('SELECT p."Team_Initials", COUNT(m."MatchID") AS "Number of Matches",ROUND(SUM(CAST(m."Attendance" AS DECIMAL))/COUNT(m."MatchID"),3) AS "Average Attendance per Match" FROM wc_players AS p INNER JOIN wc_matches AS m ON p."MatchID"=m."MatchID" GROUP BY p."Team_Initials" ORDER BY "Number of Matches" DESC;', con=engine)
team_attend_matches_df.head(10)

,Team_Initials,Number of Matches,Average Attendance per Match
0,BRA,2311,54671.804
1,ITA,1843,50671.091
2,ARG,1715,52607.808
3,ENG,1378,47448.586
4,FRG,1364,52802.823
5,ESP,1316,44768.413
6,FRA,1297,45422.877
7,MEX,1163,48472.301
8,URU,1136,43999.686
9,NED,1118,52615.433


In [43]:
team_average_attend_df = pd.read_sql_query('SELECT p."Team_Initials", COUNT(m."MatchID") AS "Number of Matches", ROUND(SUM(CAST(m."Attendance" AS DECIMAL))/COUNT(m."MatchID"),3) AS "Average Attendance per Match" FROM wc_players AS p INNER JOIN wc_matches AS m ON p."MatchID"=m."MatchID" GROUP BY p."Team_Initials" ORDER BY "Average Attendance per Match" DESC;', con=engine)
team_average_attend_df.head(10)

,Team_Initials,Number of Matches,Average Attendance per Match
0,SLV,126,60994.635
1,RUS,204,57731.078
2,ECU,230,54987.700
3,BRA,2311,54671.804
4,CIV,207,54649.111
5,BIH,69,54416.000
6,KSA,291,53906.405
7,SCG,69,53666.667
8,GHA,276,53079.167
9,FRG,1364,52802.823


In [44]:
player_attend_matches_df = pd.read_sql_query('SELECT p."Player_Name", COUNT(m."MatchID") AS "Number of Matches", ROUND(SUM(CAST(m."Attendance" AS DECIMAL))/COUNT(m."MatchID"),3) AS "Average Attendance per Match" FROM wc_players AS p INNER JOIN wc_matches AS m ON p."MatchID"=m."MatchID" GROUP BY p."Player_Name" ORDER BY "Number of Matches" DESC;', con=engine)
player_attend_matches_df.head(10)

,Player_Name,Number of Matches,Average Attendance per Match
0,RONALDO,33,60274.061
1,KLOSE,27,56141.519
2,CAFU,26,61690.577
3,LEAO,25,51954.840
4,Sepp MAIER,25,52957.600
5,DIDA,25,49638.000
6,OSCAR,24,50305.208
7,Wladyslaw ZMUDA,24,40603.708
8,SILVA,23,54604.304
9,Giuseppe BERGOMI,23,55419.087


In [45]:
player_average_attend_df = pd.read_sql_query('SELECT p."Player_Name", COUNT(m."MatchID") AS "Number of Matches", ROUND(SUM(CAST(m."Attendance" AS DECIMAL))/COUNT(m."MatchID"),3) AS "Average Attendance per Match" FROM wc_players AS p INNER JOIN wc_matches AS m ON p."MatchID"=m."MatchID" GROUP BY p."Player_Name" ORDER BY "Average Attendance per Match" DESC;', con=engine)
player_average_attend_df.head(10)

,Player_Name,Number of Matches,Average Attendance per Match
0,ZIZINHO,6,121936.333
1,JAIR,6,121936.333
2,ADAOZINHO,6,121936.333
3,DANILO ALVIM,6,121936.333
4,MANECA,6,121936.333
5,AUGUSTO,6,121936.333
6,NORONHA,6,121936.333
7,BARBOSA,6,121936.333
8,BIGODE,6,121936.333
9,ALFREDO II,6,121936.333


In [46]:
referee_data_df = pd.read_sql_query('SELECT m."Referee", COUNT(m."MatchID") AS "Matches", SUM(p."Yellow_Card") AS "Yellow Cards", ROUND(SUM(CAST(p."Yellow_Card" AS DECIMAL))/COUNT(m."MatchID"),3) AS "Yellows per Match", SUM(p."Red_Card") AS "Red Cards", ROUND(SUM(CAST(p."Red_Card" AS DECIMAL))/COUNT(m."MatchID"),3) AS "Reds per Match", (SUM(p."Penalty")+SUM(p."Missed_Penalty")) AS "Penalties", ROUND((SUM(CAST(p."Penalty" AS DECIMAL))+SUM(CAST(p."Missed_Penalty" AS DECIMAL)))/COUNT(m."MatchID"),3) AS "Penalties per Match" FROM wc_players AS p INNER JOIN wc_matches AS m ON p."MatchID"=m."MatchID" GROUP BY m."Referee" ORDER BY "Yellows per Match" DESC;', con=engine)
referee_data_df.head(10)

,Referee,Matches,Yellow Cards,Yellows per Match,Red Cards,Reds per Match,Penalties,Penalties per Match
0,LOPEZ NIETO Antonio (ESP),46,16,0.348,2,0.043,0,0.000
1,WEGEREEF Jan (NED),46,12,0.261,0,0.000,2,0.043
2,IVANOV Valentin (RUS),138,26,0.188,4,0.029,0,0.000
3,DIAZ PALACIO Jesus (COL),88,16,0.182,3,0.034,1,0.011
4,MICHEL Lubos (SVK),230,40,0.174,2,0.009,2,0.009
5,AL GHAMDI Khalil (KSA),92,15,0.163,1,0.011,1,0.011
6,PAIRETTO Pierluigi (ITA),44,7,0.159,0,0.000,1,0.023
7,TORO RENDON John (COL),44,7,0.159,3,0.068,0,0.000
8,VASSARAS Kyros (GRE),46,7,0.152,0,0.000,0,0.000
9,ABD EL FATAH Essam (EGY),46,7,0.152,0,0.000,0,0.000
